##Auto Regressive Integrated Moving Average Algorithm

Reference:
1. https://towardsdatascience.com/stock-market-forecasting-using-time-series-c3d21f2dd37f]
2. https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

##Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

In [ ]:
!pip install pmdarima

##Reading Data

In [ ]:
dataset = pd.read_csv("/content/10-02-2018-TO-09-01-2019AXISBANKALLN.csv")

In [ ]:
dataset.info()
dataset = dataset.drop(columns = ['Series'])
dataset = dataset.drop(columns = ['Symbol'])
dataset.head(2)

##Preprocessing Data

In [ ]:
dataset['Mean'] = (dataset["Low Price"] + dataset['High Price'])/2
steps=-1
dataset_for_prediction= dataset.copy()
dataset_for_prediction['Actual']=dataset_for_prediction['Mean'].shift(steps)

In [ ]:
dataset_for_prediction=dataset_for_prediction.dropna()

In [ ]:
dataset_for_prediction['Date'] = pd.to_datetime(dataset_for_prediction['Date'])
dataset_for_prediction.index= dataset_for_prediction['Date']

In [ ]:
dataset_for_prediction["Mean"].plot(color="green", figsize=(15,4))
plt.legend(["Next Day Value", "Mean"])
plt.title('Glenmark Pharma Stocks')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc_in = MinMaxScaler(feature_range=(0, 1))

scaled_input = sc_in.fit_transform(dataset_for_prediction[['Low Price', 'High Price','Open Price', 'Close Price', "No. of Trades", "Mean"]])
scaled_input = pd.DataFrame(scaled_input)
X = scaled_input

In [ ]:
sc_out = MinMaxScaler(feature_range=(0, 1))
scaler_output = sc_out.fit_transform(dataset_for_prediction[["Actual"]])
scaler_output =pd.DataFrame(scaler_output)
y=scaler_output

In [ ]:
X.rename(columns={0:"Low Price", 1:"High Price", 2:"Open Price", 3:"Close Price", 4:"Volume", 5:"Mean"}, inplace=True)
X.index = dataset_for_prediction.index
X.head(2)

In [ ]:
y.rename(columns={0:'Stock Price next day'}, inplace= True)
y.index=dataset_for_prediction.index
y.head(2)

In [ ]:
train_size=int(len(dataset) *0.9)
test_size = int(len(dataset)) - train_size
train_X, train_y = X[:train_size].dropna(), y[:train_size].dropna()
test_X, test_y = X[train_size:].dropna(), y[train_size:].dropna()

##Decomposing Time Series into Trend and Seasonal

In [ ]:
import statsmodels.api as sm
seas_d=sm.tsa.seasonal_decompose(X["Mean"],model="add",freq=30);
fig=seas_d.plot()
fig.set_figheight(4)
plt.show()

##Creating ACF and PACF plots

In [ ]:
y_test=y['Stock Price next day'][:train_size].dropna()

fig,ax= plt.subplots(2,1, figsize=(10,5))
fig=sm.tsa.graphics.plot_acf(y_test, lags=50, ax=ax[0])
fig=sm.tsa.graphics.plot_pacf(y_test, lags=50, ax=ax[1])
plt.show()

##Finding best p,q,d paramters using autoarima

In [ ]:
from pmdarima.arima import auto_arima
step_wise=auto_arima(train_y, exogenous= train_X, start_p=1, start_q=1, max_p=7, max_q=7, d=1, max_d=7, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

In [ ]:
step_wise.summary()
order1=step_wise.get_params()['order']

##Training Model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model= SARIMAX(train_y, exog=train_X,  order=order1,  enforce_invertibility=False, enforce_stationarity=False)

In [ ]:
results= model.fit()


##Getting predictions and visualizing the results

In [ ]:
predictions= results.predict(start =train_size, end=train_size+test_size+(steps)-1,exog=test_X)


In [ ]:
forecast_1= results.forecast(steps=test_size-1, exog=test_X)


In [ ]:
act= pd.DataFrame(scaler_output.iloc[train_size:, 0])


In [ ]:
predictions=pd.DataFrame(predictions)
predictions.reset_index(drop=True, inplace=True)
predictions.index=test_X.index
predictions['Actual'] = act['Stock Price next day']
predictions.rename(columns={0:'Pred'}, inplace=True)

In [ ]:
predictions['Actual'].plot(legend=True, color='blue')
predictions['Pred'].plot(legend=True, color='red', figsize=(20,8))
plt.title('Quick Heal 2018-20')


##Calculating Errors

In [ ]:
error=predictions['Pred']-predictions['Actual']
N=predictions['Pred'].shape[0]
error=np.absolute(error)
error1=error/predictions['Actual']


In [ ]:
mean_abs_error=error.sum()/N
mean_sq_error=(error*error).sum()/N
mean_abs_error1=error1.sum()/N
mean_sq_error1=(error1*error1).sum()/N
print (mean_abs_error)
print (mean_sq_error)
print (mean_abs_error1)
print (mean_sq_error1)

In [ ]:
import statsmodels
k=np.column_stack((predictions['Pred'],predictions['Actual']))
statsmodels.tsa.stattools.grangercausalitytests(k,0,False,True)
print(statsmodels.tsa.stattools.grangercausalitytests(k,1,True,True))


In [ ]:
k1=np.corrcoef(predictions['Pred'],predictions['Actual'])
print (k1[0][1])
print (mean_abs_error1*100)